### Import Libraries


---



In [1]:
import tensorflow as tf
import numpy as np
import os
import time

### Check GPU functionality

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
tf.test.gpu_device_name()
!nvidia-smi

Fri Feb 11 16:47:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   40C    P2    45W / 216W |   1110MiB /  8192MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Download Light Novel Title dataset


In [5]:
filepath = tf.keras.utils.get_file('light_novel_titles_clean_v2.txt', 'https://drive.google.com/uc?export=download&id=13ExvJcOr0l8LZD1gHCiYJn2C_pKx8v1D')

### Check data content

In [6]:
text = open(filepath, 'rb').read().decode(encoding='utf-16')
print(f'Length of text: {len(text)} characters')

Length of text: 60422 characters


In [7]:
# Check first 250 characters
print(text[:250])

"Thereafter of an Exiled Magician ~Somehow My Engagement Was Cancelled, and I Was Driven Out, so I Will Stop Living the Tough Aristocrat Life, and Live Leisurely in a Distant, Foreign, Developing Village~"	
Mahoutsukai no Konyakusha (LN)	
Exiled Pr


In [8]:
# Check amount of unique characters
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')


118 unique characters


## Text Processing

### Text Vectorization


In [9]:
chars_to_ids = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
ids_to_chars = tf.keras.layers.StringLookup(vocabulary=chars_to_ids.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(ids_to_chars(ids), axis =-1)

### Training Examples

In [10]:
all_ids = chars_to_ids(tf.strings.unicode_split(text,'UTF-8'))
all_ids

<tf.Tensor: shape=(60422,), dtype=int64, numpy=array([ 6, 50, 66, ...,  1,  3,  2], dtype=int64)>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
for ids in ids_dataset.take(10):
  print(ids_to_chars(ids).numpy().decode('UTF-8'))

"
T
h
e
r
e
a
f
t
e


In [13]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length)+1


In [14]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
  print(ids_to_chars(seq))

tf.Tensor(
[b'"' b'T' b'h' b'e' b'r' b'e' b'a' b'f' b't' b'e' b'r' b' ' b'o' b'f'
 b' ' b'a' b'n' b' ' b'E' b'x' b'i' b'l' b'e' b'd' b' ' b'M' b'a' b'g'
 b'i' b'c' b'i' b'a' b'n' b' ' b'~' b'S' b'o' b'm' b'e' b'h' b'o' b'w'
 b' ' b'M' b'y' b' ' b'E' b'n' b'g' b'a' b'g' b'e' b'm' b'e' b'n' b't'
 b' ' b'W' b'a' b's' b' ' b'C' b'a' b'n' b'c' b'e' b'l' b'l' b'e' b'd'
 b',' b' ' b'a' b'n' b'd' b' ' b'I' b' ' b'W' b'a' b's' b' ' b'D' b'r'
 b'i' b'v' b'e' b'n' b' ' b'O' b'u' b't' b',' b' ' b's' b'o' b' ' b'I'
 b' ' b'W' b'i'], shape=(101,), dtype=string)


In [15]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'"Thereafter of an Exiled Magician ~Somehow My Engagement Was Cancelled, and I Was Driven Out, so I Wi'
b'll Stop Living the Tough Aristocrat Life, and Live Leisurely in a Distant, Foreign, Developing Villag'
b'e~"\t\r\nMahoutsukai no Konyakusha (LN)\t\r\nExiled Prince Without Skills: Infinite Growth in a Mysterious '
b'Dungeon\t\r\nReincarnation Of The Strongest Sword God\t\r\nHard Work Will Never Betray Me (LN)\t\r\nThe Protag'
b'onists Are Murdered by Me\t\r\nInfinite Lucky Emperor\t\r\nStarting from the Dragon Tribe\t\r\nArs Goetia \xe2\x80\x93 Th'


In [16]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [17]:
dataset = sequences.map(split_input_target)

In [18]:
for input_example, target_example in dataset.take(1):
  print("Input:", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input: b'"Thereafter of an Exiled Magician ~Somehow My Engagement Was Cancelled, and I Was Driven Out, so I W'
Target: b'Thereafter of an Exiled Magician ~Somehow My Engagement Was Cancelled, and I Was Driven Out, so I Wi'


### Training batch creation

In [19]:
# Batch size
BATCH_SIZE = 64
# Buffer size for dataset shuffle
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build Model

In [20]:
# Length of vocabulary in chars
vocab_size = len(vocab)

# Embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [23]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
                                    
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [24]:
model = MyModel(
    # Be sure vocabulary size matches the 'StringLookup' layers.
    vocab_size=len(chars_to_ids.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


## Model test

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "(batch_size, sequence_length, vocab_size)")

(64, 100, 119) (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  30464     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  121975    
                                                                 
Total params: 4,090,743
Trainable params: 4,090,743
Non-trainable params: 0
_________________________________________________________________


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [28]:
sampled_indices

array([ 64,  26,  14,  27,  62,  51,  83, 117,  67,   3,  40,  62,  23,
        77,  40,  86,  18,  62,  22,  81,  16,  86,  99,   1,  10, 115,
        53,   4,  14,  39, 118,   7,  48,  19,  70, 100,  85,  19,  52,
        19,  61, 101,  42,  43,  21,   4,  79,  46,  23,  62,  71,  20,
        90,  98,  89,   0,  81, 104,  64,  40,  21, 102,  84,  63,  36,
        15,  32,  76,  75, 102,  59,   3,  88,  13,   6,  29,  33,  44,
        62,  21,  36, 112,  37,  50,   4, 117, 113,  90, 112,  69,  53,
        29,  90,  79, 105,  37,  99,  40,  84,  47], dtype=int64)

In [29]:
# Decode untrained model prediction
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'level Store Manager\t\r\n"The Dismissed Royal Magician is Returning Home ~The Country is a Facing Crisi'

Next Char Predictions:
 b'f8,9dUy\xef\xbc\x9ai\rJd5sJ\xc2\x810d4w.\xc2\x81\xc5\x93\t(\xe3\x80\x91W ,I\xef\xbd\x9e%R1l\xc5\xa0~1V1c\xcb\x9cLM3 uP5dm2\xc2\xa0\xc5\x8d\xc2\x9d[UNK]w\xe2\x80\x98fJ3\xe2\x80\x93zeF-Brq\xe2\x80\x93a\r\xc2\x90+"=CNd3F\xe3\x80\x8cGT \xef\xbc\x9a\xe3\x80\x8d\xc2\xa0\xe3\x80\x8ckW=\xc2\xa0u\xe2\x80\x99G\xc5\x93JzQ'


## Model Training

### Optimizer and Loss Function

In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, "(batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)


Prediction shape:  (64, 100, 119) (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.7796, shape=(), dtype=float32)


In [32]:
tf.exp(example_batch_mean_loss).numpy()

119.05674

In [33]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [34]:
# Checkpoint save directory
checkpoint_dir = './training_checkpoints'

# Name of checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Start training

In [35]:
EPOCHS=100

In [36]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
9/9 [==============================] - 4s 115ms/step - loss: 4.8178
Epoch 2/100
9/9 [==============================] - 1s 111ms/step - loss: 4.2993
Epoch 3/100
9/9 [==============================] - 1s 113ms/step - loss: 3.6609
Epoch 4/100
9/9 [==============================] - 1s 113ms/step - loss: 3.3490
Epoch 5/100
9/9 [==============================] - 3s 305ms/step - loss: 3.1359
Epoch 6/100
9/9 [==============================] - 1s 109ms/step - loss: 2.8754
Epoch 7/100
9/9 [==============================] - 1s 105ms/step - loss: 2.6746
Epoch 8/100
9/9 [==============================] - 1s 107ms/step - loss: 2.5696
Epoch 9/100
9/9 [==============================] - 1s 105ms/step - loss: 2.5001
Epoch 10/100
9/9 [==============================] - 1s 105ms/step - loss: 2.4506
Epoch 11/100
9/9 [==============================] - 1s 107ms/step - loss: 2.4040
Epoch 12/100
9/9 [==============================] - 1s 107ms/step - loss: 2.3634
Epoch 13/100
9/9 [===================

In [37]:
TEMP = 0.5

In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, ids_to_chars, chars_to_ids, temperature=TEMP):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.ids_to_chars = ids_to_chars
    self.chars_to_ids = chars_to_ids

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.chars_to_ids(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(chars_to_ids.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.chars_to_ids(input_chars).to_tensor()

    # Run model
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply prediction mask: prevent UNK generation.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert token ids to characters
    predicted_chars = self.ids_to_chars(predicted_ids)

    # Return characters and model state.
    return predicted_chars, states

In [41]:
one_step_model = OneStep(model, ids_to_chars, chars_to_ids)

In [44]:
SEED = "Is this a light novel title?"

In [45]:
start = time.time()
states = None
# Start String
next_char = tf.constant([SEED])
result = [next_char]

for n in range(2000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  # TODO: Check if generated title is in source text, if yes -> regenerate
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Is this a light novel title?	
"Tee Onl OwMy Fett Commonites"	
Baby is Created Go For Much, is a Gothure Tare	
I Have A Super USB Drive	
The Kingdom of Endless Magic!	
Reincarnated as an Aristocrat with the First Step	
Cheat Skills in Another World and Live by Relying on Other's World	
The Failure Red Mage is Unmatched	
Without Elfays The World	
King Artule ~ Thes the Hero and the Saintess"	
My Daughter Grew Until Hist Disch	
A SIcueme the Harat Skold	
The Grast Emperor as Husband	
The Struggle of Returning to Be a Horror Movie Boss	
Never Thought You'd Be This Kind of Hero!	
A Mid-leve Sprongest After Spamming the 100 Million Years Buttong of the Villain	
The Novel's Extra	
Mahou Gakuen no Taizai Majutsushi	
She's the Older Sister of the Possessed With Me After I Saved Him	
Omake no Tensei-sha	
I'm Really Not The Demon God's Lacker	
"The Dragon Thio Loves in Another World	
"I Became a Regulatory Official for the Poactsu"	
Thr Poorion Life In The Regent	
"Hen na Ryu to Moto Yuusha Party

## Export Generator

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant([SEED])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))